In [ ]:
import random
import numpy as np
import matplotlib
import time
from numba import jit
import multiprocessing as mp
from matplotlib import pyplot as plt

In [ ]:
@jit(nopython=True)
def demand(p1, p2):
    if p1 < p2:
        d = 1 - p1
    elif p1 == p2:
        d = 0.5 * (1 - p1)
    else:
        d = 0
    return d

# Profitfunktion

In [ ]:
@jit(nopython=True)
def profit(p1, p2):
    return (p1 * demand(p1, p2))

In [ ]:
P = np.array([0, 1/6, 2/6, 3/6, 4/6, 5/6, 1]) # price array

# Hjælpefunktion...

In [ ]:
np.random.seed(123)

@jit(nopython=True)
def epsilon_greedy(Qtable, epsilon, uniform, state: int, price_grid: np.ndarray) -> int: 
    N = len(price_grid)
    assert Qtable.shape[0] == N, "Qtable must have the same number of rows as there are prices in the grid"
    assert Qtable.shape[1] == N, "Qtable must have the same number of columns as there are prices in the grid"
    assert state < N, "state must be a valid index in the price grid"
    assert state >= 0, "state must be a valid index in the price grid"

    # Draw action 
    if uniform < epsilon:
        price_index = np.random.choice(N)
        #print("random")
    else:
        price_index = np.argmax(Qtable[:, state])
        #print("ikke random")
    return price_index

In [ ]:
@jit(nopython=True)
def Qfunction(price_grid, period, delta, alpha, theta):
    # Initialize prices and Q-tables
    price_index_i = np.random.choice(len(price_grid)) 
    price_index_j = np.random.choice(len(price_grid)) 

    #state = np.random.choice(len(price_grid))

    Qtable_i = np.zeros((len(price_grid), len(price_grid)))
    Qtable_j = np.zeros((len(price_grid), len(price_grid)))

    profit_1 = np.zeros(int(period))
    profit_2 = np.zeros(int(period))

    total_profit = np.zeros(period)

    epsilons = (1 - theta)**np.arange(period + 1)
    uniforms = np.random.uniform(0, 1, (period + 1, 2))

    for t in range(1, period + 1):
        #epsilon = (1 - theta)**t

        # figure out who's turn it is
        if t % 2 == 0:
            # player 2 is the responder
            # set up pointers to the state and the Q table 
            state = price_index_j # the most recent draw of player 1's price
            Qtable_i = Qtable_j 
            Qtable_j = Qtable_i
            player_index = 1
            opponent_index = 0
        else:
            # player 1 is the responder
            state = price_index_j
            Qtable_i = Qtable_i
            Qtable_j = Qtable_j
            player_index = 0
            opponent_index = 1

        # draw i's action 
        # (actually, player i's action is already simulated by player j in the previous period)
        price_index_i = epsilon_greedy(Qtable_i, epsilons[t], uniforms[t, player_index], state=state, price_grid=price_grid)

        # current period
        decision = price_grid[price_index_i]
        state_i_responds_to = price_grid[state]
        profit_i_current_period = profit(decision, state_i_responds_to)

        # next period 
        state_next_period = price_index_i # next_period's state is today's price
        price_index_j_next_period = epsilon_greedy(Qtable_j, epsilons[t], uniforms[t, opponent_index], state=state_next_period, price_grid=price_grid)

        price_j_next_period = price_grid[price_index_j_next_period]
        price_i_next_period = price_grid[price_index_i] # unchanged price, it's not i's turn 
        profit_i_next_period = profit(price_i_next_period, price_j_next_period)

        max_Q = np.max(Qtable_i[:, price_index_j_next_period])
        continuation_value = max_Q 

        new_estimate = profit_i_current_period + delta * profit_i_next_period + delta**2 * continuation_value

        # Update
        prev_estimate = Qtable_i[price_index_i, price_index_j]
        Qtable_i[price_index_i, price_index_j] = (1 - alpha) * prev_estimate + alpha * new_estimate

        # Update for the next iteration: Use the simulated next period's action as the actual action for the opponent
        if t % 2 == 0:
        # If current period is Player 2's turn, update Player 1's price for the next period
            price_index_j = price_index_i
            price_index_i = price_index_j_next_period
            profit_1[t-1] = profit(state_i_responds_to, decision)
            profit_2[t-1] = profit_i_current_period
        else:
        # If current period is Player 1's turn, update Player 2's price for the next period
            price_index_j = price_index_i
            price_index_i = price_index_j_next_period
            profit_1[t-1] = profit_i_current_period
            profit_2[t-1] = profit(state_i_responds_to, decision)


        total_profit[t-1] = profit_i_current_period

    return total_profit, profit_1, profit_2


@jit(nopython=True)
def Simulations(sim, price, period, delta, alpha, theta):
    total_profit_sim = np.zeros((sim, period))
    profit_1_sim = np.zeros((sim, period))
    profit_2_sim = np.zeros((sim, period))
    avg_profit = np.zeros(sim)
    avg_profit_1 = np.zeros(sim)
    avg_profit_2 = np.zeros(sim)

    for i in range(sim):
        total_profit_array, profit_1_array, profit_2_array = Qfunction(price, period, delta, alpha, theta)
        total_profit_sim[i] = total_profit_array
        profit_1_sim[i] = profit_1_array
        profit_2_sim[i] = profit_2_array
        avg_profit_1[i] = np.mean(profit_1_array[-1000:])
        avg_profit_2[i] = np.mean(profit_2_array[-1000:])
        avg_profit[i] = np.mean(total_profit_array[-1000:])

    return total_profit_sim, avg_profit_1, avg_profit_2, profit_1_sim, profit_2_sim

# Time calculation for 1000 simulations

In [ ]:
start_time  = time.time()

np.random.seed(128)
total_profit_plot, avg_profit_1_plot, avg_profit_2_plot, profit_total_1, profit_total_2 = Simulations(1000, P, 500000, 0.95, 0.3, 0.0000276306)

end_time = time.time()

elapsed_time = end_time - start_time

print("Time taken to run 1000 simulations:", elapsed_time, "seconds")